In [61]:
import requests
import re

import pandas as pd
df = pd.DataFrame(columns=['source', 'news', 'linc', 'date'])

import pymongo as pm
from user_agent import generate_user_agent

from lxml import html

In [109]:
def get_atr_mailru(u):
    global df
    req = requests.get(u, headers = {"User-Agent":generate_user_agent()}).text
    root = html.fromstring(req)
    
    source = root.xpath("//span[contains(text(),'{}')]".format('"'))
    source = source if len(source) > 0 else root.xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']")
    source = source[0].text if len(source) > 0 else 'не подписано'
    news = root.xpath("//h1[@class='hdr__inner']")[0].text
    linc = u
    date = root.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime | //time[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    
    return [source,news,linc,date]
    

def get_news_mailru():
    global df
    url = 'https://mail.ru/?from=m'
    req = requests.get(url, headers = {"User-Agent":generate_user_agent()}).text
    
    root = html.fromstring(req)

    for item in root.xpath("//div[@class='list'][1]/a/@href"):
        print(item)
        df.loc[len(df)+1] = get_atr_mailru(item)
    print(df)

In [293]:
def get_atr_lentaru(u):
    global df
    print(u)
    req = requests.get(u, headers = {"User-Agent":generate_user_agent()}).text
    root = html.fromstring(req)
    
    source = 'lenta.ru'
    news = root.xpath("//h1[@class='b-topic__title'] | //div[@class='premial-header__title'] | //h1[@class='brief-header__title']")[0].text
    linc = u
    try:
        date = root.xpath("//div[@class='b-topic__info']/time/@datetime | //div[@class='premial-header__date']")[0]
    except:
        date = "не определена"
    
    return [source,news,linc,date]
    

def get_news_lentaru():
    global df
    url = 'https://lenta.ru'
    req = requests.get(url, headers = {"User-Agent":generate_user_agent()}).text

    root = html.fromstring(req)

    for item in root.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item']/a/@href | //section[@class='b-layout js-layout b-layout_main']//div[@class='titles']//a/@href"):
        ext = re.findall('(https:\/\/[\w\. _\d]+)\/|\/(extlink)\/', item)

        if len(ext) > 0:
            r1 = root.xpath(f"//a[@href='{item}']/text()")
            d1 = root.xpath(f"//a[@href='{item}']/time/@datetime")
            try:
                df.loc[len(df)+1] = [ext[0],r1,item,d1]
            except:
                df.loc[len(df)+1] = [ext[0],"Надо поработать со сторонними сайтами",item, "Дата"]
        else:
            df.loc[len(df)+1] = get_atr_lentaru(url+ item)

In [ ]:
get_news_mailru()
get_news_lentaru()

In [292]:
df.head(10)


,source,news,linc,date
1,lenta.ru,Актер оголил «кукольный» пах ради модной съемки,https://lenta.ru/news/2019/11/22/petedavidson/,2019-11-22T16:57:18+03:00
2,lenta.ru,Появилось видео с собирающимся расстрелять пол...,https://lenta.ru/news/2019/11/22/siktivkar/,2019-11-22T16:56:00+03:00
3,lenta.ru,Актер оголил «кукольный» пах ради модной съемки,https://lenta.ru/news/2019/11/22/petedavidson/,2019-11-22T16:57:00+03:00
4,lenta.ru,Появилось видео с собирающимся расстрелять пол...,https://lenta.ru/news/2019/11/22/siktivkar/,2019-11-22T16:56:00+03:00
5,lenta.ru,Тарасова объяснила провал Загитовой,https://lenta.ru/news/2019/11/22/tarasova/,2019-11-22T17:24:05+03:00
6,lenta.ru,В России стало больше случаев избиений военнос...,https://lenta.ru/news/2019/11/22/assault/,2019-11-22T17:23:27+03:00
7,lenta.ru,Россиянам стало рекордно невыгодно нести деньг...,https://lenta.ru/news/2019/11/22/low/,2019-11-22T17:21:00+03:00
8,lenta.ru,Украинцев задержали за отправку дроном марихуа...,https://lenta.ru/news/2019/11/22/kannabis/,2019-11-22T17:20:00+03:00
9,lenta.ru,Запуск паромов между Крымом и Турцией опровергли,https://lenta.ru/news/2019/11/22/paromanet/,2019-11-22T17:16:00+03:00
10,lenta.ru,Актер оголил «кукольный» пах ради модной съемки,https://lenta.ru/news/2019/11/22/petedavidson/,2019-11-22T16:57:00+03:00
